<a href="https://colab.research.google.com/github/andreaaraldo/machine-learning-for-networks/blob/master/08.predictive-maintenance/Predictive-maintenance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

!pip install scikit-survival



     |████████████████████████████████| 1.8MB 8.2MB/s 
     |████████████████████████████████| 11.7MB 21.8MB/s 
     |████████████████████████████████| 952kB 25.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.0MB 40.2MB/s 
     |████████████████████████████████| 163kB 52.0MB/s 
  Using cached https://files.pythonhosted.org/packages/03/27/e35e7c6e6a52fab9fcc64fc2b20c6b516eba930bb02b10ace3b38200d3ab/numpy-1.18.4-cp36-cp36m-manylinux1_x86_64.whl
     |████████████████████████████████| 153kB 53.9MB/s 
     |████████████████████████████████| 10.4MB 46.4MB/s 
  Using cached https://files.pythonhosted.org/packages/dc/29/162476fd44203116e7980cfbd9352eef9db37c49445d1fec35509022f6aa/scipy-1.4.1-cp36-cp36m-manylinux1_x86_64.whl
     |████████████████████████████████| 6.7MB 44.3MB/s 
     |████████████████████████████████| 3.6MB 41.5MB/s 
     |████████████████████████████

In [0]:
#import sksurv 
#from sksurv.linear_model import CoxPHSurvivalAnalysis
#from sksurv.metrics import concordance_index_censored

In [2]:
! wget https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/08.predictive-maintenance/dataset/transformed/test_set.csv
! wget https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/08.predictive-maintenance/dataset/transformed/training_set.csv

--2020-05-19 21:36:11--  https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/08.predictive-maintenance/dataset/transformed/test_set.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4397297 (4.2M) [text/plain]
Saving to: ‘test_set.csv’

test_set.csv        100%[===================>]   4.19M  --.-KB/s    in 0.08s   

2020-05-19 21:36:12 (50.3 MB/s) - ‘test_set.csv’ saved [4397297/4397297]

--2020-05-19 21:36:13--  https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/08.predictive-maintenance/dataset/transformed/training_set.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubuserc

In [0]:
df_trn = pd.read_csv("training_set.csv")
df_test = pd.read_csv("training_set.csv")

In [4]:
df_trn.head()

,setting1,setting2,setting3,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,remaining_duration,failure_observed
0,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,1.0,0.726248,0.242424,0.109755,0.0,0.369048,0.633262,0.205882,0.199608,0.363986,0.0,0.333333,0.0,0.0,0.713178,0.724662,191,1
1,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,1.0,0.628019,0.212121,0.100242,0.0,0.380952,0.765458,0.279412,0.162813,0.411312,0.0,0.333333,0.0,0.0,0.666667,0.731014,190,0
2,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,1.0,0.710145,0.272727,0.140043,0.0,0.250000,0.795309,0.220588,0.171793,0.357445,0.0,0.166667,0.0,0.0,0.627907,0.621375,189,0
3,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,1.0,0.740741,0.318182,0.124518,0.0,0.166667,0.889126,0.294118,0.174889,0.166603,0.0,0.333333,0.0,0.0,0.573643,0.662386,188,1
4,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,1.0,0.668277,0.242424,0.149960,0.0,0.255952,0.746269,0.235294,0.174734,0.402078,0.0,0.416667,0.0,0.0,0.589147,0.704502,187,1
